# Data Sorting #

In [1]:
import pandas as pd
import numpy as np
import math
import time
from statsmodels.discrete.discrete_model import Probit
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

df = pd.io.stata.read_stata('20160213_data/finished_do.dta')

# Reproduction of results # 

## Table 2 - The impact of race on likelihood of acceptance ##

In [2]:
df_model = df[['yes','guest_black','name_by_city', 'host_gender_M', 'host_race_black']].dropna()
df_model3 = df[['yes','guest_black','name_by_city', 'host_gender_M', 'host_race_black', 'multiple_listings', 'shared_property', 'ten_reviews', 'log_price']].dropna()

model = smf.ols('yes ~ guest_black', data=df_model)
result1 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model['name_by_city']})

model = smf.ols('yes ~ guest_black + host_race_black + host_gender_M', data=df_model)
result2 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model['name_by_city']})

model = smf.ols('yes ~ guest_black + host_race_black + host_gender_M + multiple_listings + shared_property + ten_reviews + log_price', data=df_model3)
result3 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model3['name_by_city']})


tble = summary_col([result1, result2, result3], stars=True, float_format='%.2f', regressor_order=['guest_black', 'host_race_black', 'host_gender_M'], info_dict={'N':lambda x: "{0:d}".format(int(x.nobs))})
tble

,yes I,yes II,yes III
guest_black,-0.08***,-0.08***,-0.09***
,(0.02),(0.02),(0.02)
host_race_black,,0.07***,0.09***
,,(0.02),(0.02)
host_gender_M,,-0.05***,-0.05***
,,(0.01),(0.01)
Intercept,0.49***,0.50***,0.76***
,(0.01),(0.01),(0.07)
R-squared,0.01,0.01,0.04
,0.01,0.01,0.04


## Table 3: Race Gap by Race of the Host, across all hosts, then across male and female hosts ##

In [3]:
df['guest_host_black'] = df['guest_black'] * df['host_race_black']
df_model = df[['yes','guest_black','name_by_city', 'guest_host_black', 'host_race_black', 'host_gender_M', 'host_gender_F']].dropna()

df_model_gender = df_model
model = smf.ols('yes ~ guest_black + host_race_black + guest_host_black', data=df_model_gender)
result1 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model_gender['name_by_city']})

df_model_gender = df_model[df_model['host_gender_M'] == 1]
model = smf.ols('yes ~ guest_black + host_race_black + guest_host_black', data=df_model_gender)
result2 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model_gender['name_by_city']})

df_model_gender = df_model[df_model['host_gender_F'] == 1]
model = smf.ols('yes ~ guest_black + host_race_black + guest_host_black', data=df_model_gender)
result3 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model_gender['name_by_city']})

df_model_gender = df_model[(df_model['host_gender_F'] != 1) & (df_model['host_gender_M'] != 1)]
model = smf.ols('yes ~ guest_black + host_race_black + guest_host_black', data=df_model_gender)
result4 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model_gender['name_by_city']})


tble2 = summary_col([result1, result2, result3, result4], stars=True, float_format='%.2f', 
                   regressor_order=['guest_black', 'host_race_black', 'guest_host_black'], 
                   model_names=('All hosts', 'Male hosts', 'Female hosts', 'Other hosts'),  
                   info_dict={'N':lambda x: "{0:d}".format(int(x.nobs))})
tble2

,All hosts,Male hosts,Female hosts,Other hosts
guest_black,-0.08***,-0.09***,-0.09***,-0.07**
,(0.02),(0.02),(0.02),(0.03)
host_race_black,0.06**,0.19***,-0.00,0.03
,(0.03),(0.05),(0.04),(0.09)
guest_host_black,0.01,-0.11,0.11*,-0.06
,(0.05),(0.08),(0.06),(0.14)
Intercept,0.48***,0.44***,0.50***,0.50***
,(0.01),(0.02),(0.02),(0.02)
R-squared,0.01,0.01,0.01,0.00
,0.01,0.02,0.01,0.00


## Table 4. Proportion of Positive Responses by Race and Gender ##

In [4]:
# Format race/gender interaction columns as naming is inconsistent
df['host_male'] = df['host_gender_M']
df['host_female'] = df['host_gender_F']
df['guest_race_black']= df['guest_black']
df['guest_race_white'] = df.guest_black.apply(lambda x: 1 if x==0 else 0)

for gender in ['female', 'male']:
    for race in ['white', 'black']:
        for side in ['guest', 'host']:
            df[side+'_'+gender+'_'+race] = df.apply(lambda x: 1 if x[side+'_'+gender] == 1 and x[side+'_race_'+race] ==1 else 0, axis=1)

df['no'] = df.yes.apply(lambda x: 0 if x==1 else 1)
host_combinations = ['host_male_white', 'host_male_black','host_female_white', 'host_female_black']
guest_combinations = ['guest_female_white', 'guest_female_black', 'guest_male_white', 'guest_male_black']

# Sum of positive responses
table1 = pd.pivot_table(df, values=guest_combinations, index=host_combinations,
                   columns=['yes'], aggfunc=np.sum)
# Sum of negative responses
table2 = pd.pivot_table(df, values=guest_combinations, index=host_combinations,
                   columns=['no'], aggfunc=np.sum)
# Response rate
table3 = table1 / (table1+table2)

# Formatting
table3.drop(columns=table3.columns[::2], inplace=True)
table3 = table3.iloc[1:]
table3.reset_index(inplace=True)
table3.reindex(host_combinations)
host_combinations.reverse()
table3.index = host_combinations
table3 = table3.iloc[:,4:]
table3.columns = table3.columns.get_level_values(0)
table3 = table3[['guest_male_white', 'guest_male_black','guest_female_white', 'guest_female_black']]
host_combinations.reverse()
table3.reindex(host_combinations)
pd.options.display.float_format = '{:,.2f}'.format

table3

,guest_male_white,guest_male_black,guest_female_white,guest_female_black
host_female_black,0.43,0.38,0.53,0.59
host_female_white,0.46,0.35,0.49,0.44
host_male_black,0.64,0.40,0.59,0.43
host_male_white,0.42,0.35,0.49,0.32


## Table 5. Are Effects Driven by Host Characteristics? ##


In [5]:
df['shared_guest_black'] = df['shared_property'] * df['guest_black']
df['multiple_black'] = df['multiple_listings'] * df['guest_black']
df['ten_reviews_black'] = df['ten_reviews'] * df['guest_black']
df['young_black'] = df['young'] * df['guest_black']
df['any_black_gb'] = df['any_black'] * df['guest_black']


# This extra df is created to jointly drop NA values including 'name_by_city'. Otherwise, as 
# 'name_by_city' is only called in the .fit() function, the length of it does not match the
# exogenous variables from the model where NA values where already dropped.
df_model = df[['yes','guest_black','name_by_city', 'shared_property', 'shared_guest_black', 
               'multiple_listings', 'multiple_black', 'ten_reviews', 'ten_reviews_black',
              'young', 'young_black', 'any_black', 'any_black_gb']].dropna()


model = smf.ols('yes ~ guest_black + shared_property + shared_guest_black', data=df_model)
result1 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model['name_by_city']})

model = smf.ols('yes ~ guest_black + multiple_listings + multiple_black', data=df_model)
result2 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model['name_by_city']})

model = smf.ols('yes ~ guest_black + ten_reviews + ten_reviews_black', data=df_model)
result3 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model['name_by_city']})

model = smf.ols('yes ~ guest_black + young + young_black', data=df_model)
result4 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model['name_by_city']})

model = smf.ols('yes ~ guest_black + any_black + any_black_gb', data=df_model)
result5 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model['name_by_city']})

tble5 = summary_col([result1, result2, result3, result4, result5], stars=True, float_format='%.2f', 
                   regressor_order=['guest_black','shared_property', 'shared_guest_black', 
                        'multiple_listings', 'multiple_black', 'ten_reviews', 'ten_reviews_black',
                        'young', 'young_black', 'any_black', 'any_black_gb'],  
                   info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),'R2_adj':lambda x: "{:.3f}".format(x.rsquared_adj)})
tble5

,yes I,yes II,yes III,yes IIII,yes IIIII
guest_black,-0.07***,-0.08***,-0.09***,-0.08***,-0.09***
,(0.02),(0.02),(0.02),(0.02),(0.02)
shared_property,0.00,,,,
,(0.01),,,,
shared_guest_black,-0.02,,,,
,(0.03),,,,
multiple_listings,,0.10***,,,
,,(0.02),,,
multiple_black,,-0.00,,,
,,(0.03),,,


## Table 6. Are Effects Driven by Location Characteristics? ##

In [6]:
df['guest_black_price_median'] = df['guest_black'] * df['price_median']
df['guest_black_pop_black'] = df['guest_black'] * df['black_proportion']
df['guest_black_tract_listings'] = df['guest_black'] * df['tract_listings']
df['guest_black_pr_filled'] = df['guest_black'] * df['pr_filled']

df_model = df[['yes', 'name_by_city', 'guest_black','price_median', 'guest_black_price_median']].dropna()
model = smf.ols('yes ~ guest_black + price_median + guest_black_price_median', data=df_model)
result1 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model['name_by_city']})

df_model = df[['yes', 'name_by_city', 'guest_black', 'black_proportion', 'guest_black_pop_black']].dropna()
model = smf.ols('yes ~ guest_black + black_proportion + guest_black_pop_black', data=df_model)
result2 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model['name_by_city']})

df_model = df[['yes', 'name_by_city', 'guest_black','tract_listings', 'guest_black_tract_listings']].dropna()
model = smf.ols('yes ~ guest_black + tract_listings + guest_black_tract_listings', data=df_model)
result3 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model['name_by_city']})

df_model = df[['yes', 'name_by_city', 'guest_black','pr_filled', 'guest_black_pr_filled']].dropna()
model = smf.ols('yes ~ guest_black + pr_filled + guest_black_pr_filled', data=df_model)
result4 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model['name_by_city']})

tble6 = summary_col([result1, result2, result3, result4], stars=True, float_format='%.2f', 
                   regressor_order=['guest_black','price_median', 'guest_black_price_median', 'black_proportion', 
                        'guest_black_pop_black', 'tract_listings', 'guest_black_tract_listings', 'pr_filled',
                        'guest_black_pr_filled'],  
                   info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),'R2_adj':lambda x: "{:.3f}".format(x.rsquared_adj)})
tble6

,yes I,yes II,yes III,yes IIII
guest_black,-0.08***,-0.08***,-0.09***,-0.12**
,(0.02),(0.02),(0.02),(0.06)
price_median,-0.06***,,,
,(0.02),,,
guest_black_price_median,0.01,,,
,(0.03),,,
black_proportion,,0.05,,
,,(0.05),,
guest_black_pop_black,,0.02,,
,,(0.07),,


End of reproduction of main results.

# Own empirical analysis # 

## Randomization controls ##

### Regress each control on treatment variable ###

In [7]:
used_controls = ['host_gender_M', 'host_race_black', 'multiple_listings', 'shared_property', 'log_price', 
                     'host_race_black', 'host_gender_M', 'host_gender_F', 'shared_property', 'multiple_listings', 
                     'ten_reviews', 'young', 'price_median', 'black_proportion', 'guest_male',
                     'tract_listings', 'pr_filled', 'baltimore', 'dallas', 'los_angeles', 'sl', 'dc']
used_control_models = {}

print('{0:20} {1:>1}'.format('Control Variable', 'P-Value'))
for control in used_controls:
    model = smf.ols(control + ' ~ guest_black', data=df)
    result = model.fit()
    used_control_models[control] = result
    print('{0:20} {1:>6.3f}'.format(control + ':', result.pvalues[1]))

Control Variable     P-Value
host_gender_M:        0.896
host_race_black:      0.972
multiple_listings:    0.451
shared_property:      0.929
log_price:            0.792
host_race_black:      0.972
host_gender_M:        0.896
host_gender_F:        0.439
shared_property:      0.929
multiple_listings:    0.451
ten_reviews:          0.041
young:                0.799
price_median:         0.772
black_proportion:     0.919
guest_male:           0.408
tract_listings:       0.848
pr_filled:            0.899
baltimore:            0.906
dallas:               0.311
los_angeles:          0.743
sl:                   0.382
dc:                   0.505


ten_reviews is correlated at a 5% significance level.

In [8]:
used_control_models['ten_reviews'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            ten_reviews   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     4.194
Date:                Tue, 11 Aug 2020   Prob (F-statistic):             0.0406
Time:                        21:18:19   Log-Likelihood:                -4633.9
No. Observations:                6390   AIC:                             9272.
Df Residuals:                    6388   BIC:                             9285.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.4755      0.009     53.845      0.000       0.458       0.493
guest_black     0.0256      0.013      2.048      0.041       0.001       0.050
==============================================================================
Omnibus:                    22171.105   Durbin-Watson:                   1.606
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1062.208
Skew:                           0.047   Prob(JB):                    2.21e-231
Kurtosis:                       1.005   Cond. No.                         2.62
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

However, the coefficient is "only" 0.0256 (both variables are binary). Therefore the effect is not too large. This is not good, but does not greatly damage the interpretation and validity of the whole paper.

### Regress treatment variable on all controls ###

In [9]:
formula = 'guest_black ~ '
for control in used_controls:
    formula += str(control + ' + ')
# exclude one arbitrarily chosen city (here dc), otherwise dummy variable trap/fallacy
formula = formula[:-8] 

model = smf.ols(formula, data=df)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            guest_black   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.5590
Date:                Tue, 11 Aug 2020   Prob (F-statistic):              0.923
Time:                        21:18:19   Log-Likelihood:                -4520.5
No. Observations:                6235   AIC:                             9077.
Df Residuals:                    6217   BIC:                             9198.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.5556      0.099      5.617      0.000       0.362       0.750
host_gender_M        -0.0133      0.017     -0.801      0.423      -0.046       0.019
host_race_black      -0.0011      0.026     -0.040      0.968      -0.052       0.050
multiple_listings     0.0037      0.015      0.257      0.797      -0.025       0.032
shared_property      -0.0148      0.022     -0.683      0.495      -0.057       0.028
log_price             0.0014      0.015      0.092      0.927      -0.029       0.032
host_gender_F        -0.0169      0.016     -1.087      0.277      -0.047       0.014
ten_reviews           0.0304      0.015      2.044      0.041       0.001       0.060
young                 0.0059      0.013      0.438      0.661      -0.020       0.032
price_median         -0.0115      0.021     -0.548      0.584      -0.053       0.030
black_proportion     -0.0029      0.039     -0.076      0.940      -0.079       0.073
guest_male           -0.0119      0.013     -0.934      0.350      -0.037       0.013
tract_listings        0.0004      0.001      0.446      0.656      -0.001       0.002
pr_filled            -0.0971      0.084     -1.159      0.247      -0.261       0.067
baltimore            -0.0085      0.030     -0.281      0.778      -0.068       0.051
dallas               -0.0336      0.029     -1.175      0.240      -0.090       0.022
los_angeles          -0.0122      0.018     -0.685      0.493      -0.047       0.023
sl                   -0.0390      0.044     -0.892      0.373      -0.125       0.047
==============================================================================
Omnibus:                    21655.531   Durbin-Watson:                   1.977
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1032.828
Skew:                           0.004   Prob(JB):                    5.30e-225
Kurtosis:                       1.006   Cond. No.                         269.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Only ten_reviews is significant, mirroring previous findings. Prob F-Stat very high.

## Fisher Exact Test ##

In [12]:
class FisherExactTest:
    '''Performs Fisher Exact Test Simulation
    '''
    def __init__(self):
        self.rng = np.random.default_rng()
        pass
    
    def calc_teststat(self, treatment):
        '''Calculate sum of outputs by group.

        Given the (altered) treament array whether individual is in the treatment group or not
        and given the original output array, calculate the sum of outputs for each group.

        Arguments:
            treatment(np.array): Array of the treatment variable

        Returns:
            Fisher Test Stat(int): Value of the Fisher Test Statistics given the initial output and
                the (altered) treatment population.
        '''
        treatment_output  = sum(np.multiply(treatment, self.output))
        notreatment_output = sum(np.multiply(np.logical_not(treatment), self.output))
        return treatment_output/self.n_treatment - notreatment_output/self.n_notreatment    

    def initial_arrays(self, treatment, output):
        '''Sets up original treament and output array and calculates teststat.
        
        Arguments:
            treatment(pd.Series): Column of the treatment variable
            
            output(pd.Series): Column of the output variable
            
        Returns:
            initial_teststat(int): Value of the Fisher Test Statistics given the initial output and
                the initial treatment population.
        '''
        self.initial_treatment = np.array(treatment)
        self.n_treatment = treatment.value_counts()[True] # Done here as it doesn't change and fastens iteration
        self.n_notreatment = treatment.value_counts()[False]
        self.output = np.array(output)
        
        self.initial_teststat = self.calc_teststat(self.initial_treatment)
        return self.initial_teststat
    
    def simulation(self, m=1000):
        '''Iterates over simulated permutations and returns how many were larger than initial one. '''
        count_le = 0 # Used for counting whether alternative test statistic is larger than orginal test statistic
        
        start_time = time.perf_counter()
        for i in range(0, m):
            alt_perm = self.rng.permutation(self.initial_treatment) # Creates random alternative permutation
            alt_t = self.calc_teststat(alt_perm) # Fisher test for alternative permutation
            if alt_t >= self.initial_teststat: 
                count_le += 1
        total_time = time.perf_counter() - start_time
        
        print('For m={}, {:.2f} minutes were needed.'.format(m, total_time/60))
        print('Number of simulated test statistics larger or equal to original test statistic:\t', count_le)     
        return count_le


bw = df.guest_black.apply(lambda x: True if x==0 else False if x==1 else np.NaN) # True if guest is white
df_fisher = pd.DataFrame({'white': bw, 'yes':df['yes']})
df_fisher.dropna(inplace=True)

FET = FisherExactTest()
og_test = FET.initial_arrays(df_fisher.white, df_fisher.yes)
FET.simulation(1000)

For m=1000, 0.01 minutes were needed.
Number of simulated test statistics larger or equal to original test statistic:	 0


0

## Additional analysis ##

In [13]:
for combination in host_combinations:
    model = smf.ols('yes ~ ' + combination, data=df)
    result = model.fit()
    print(result.summary())

model = smf.ols('yes ~ host_male + host_race_black + guest_black', data=df)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                    yes   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     12.00
Date:                Tue, 11 Aug 2020   Prob (F-statistic):           0.000535
Time:                        21:20:33   Log-Likelihood:                -4486.0
No. Observations:                6235   AIC:                             8976.
Df Residuals:                    6233   BIC:                             8989.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.4598      0.007     

In [14]:
for combination in guest_combinations:
    model = smf.ols('yes ~ ' + combination, data=df)
    result = model.fit()
    print(result.summary())

model = smf.ols('yes ~ guest_black + guest_male', data=df)
result = model.fit(cov_type='cluster', cov_kwds={'groups': df['name_by_city']})
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                    yes   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     36.02
Date:                Tue, 11 Aug 2020   Prob (F-statistic):           2.06e-09
Time:                        21:20:33   Log-Likelihood:                -4474.0
No. Observations:                6235   AIC:                             8952.
Df Residuals:                    6233   BIC:                             8965.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.4264      0

ValueError: The weights and list don't have the same length.

In [15]:
df.guest_male_black

0       0
1       0
2       0
3       0
4       0
       ..
6387    0
6388    0
6389    1
6390    0
6391    0
Name: guest_male_black, Length: 6392, dtype: int64

In [16]:
df.guest_male_white

0       1
1       1
2       1
3       0
4       0
       ..
6387    1
6388    0
6389    0
6390    0
6391    0
Name: guest_male_white, Length: 6392, dtype: int64

In [17]:
df_model = df[['yes', 'guest_black', 'guest_male', 'name_by_city']].dropna()
model = smf.ols('yes ~ guest_black + guest_male', data=df_model)
result = model.fit(cov_type='cluster', cov_kwds={'groups': df_model['name_by_city']})
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    yes   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     21.20
Date:                Tue, 11 Aug 2020   Prob (F-statistic):           1.95e-08
Time:                        21:20:34   Log-Likelihood:                -4465.1
No. Observations:                6235   AIC:                             8936.
Df Residuals:                    6232   BIC:                             8956.
Df Model:                           2                                         
Covariance Type:              cluster                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.5108      0.013     40.856      0.000       0.486       0.535
guest_black    -0.0802      0.015     -5.216      0.000      -0.110      -0.050
guest_male     -0.0464      0.015     -3.030      0.002      -0.076      -0.016
==============================================================================
Omnibus:                    22646.201   Durbin-Watson:                   1.954
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1004.533
Skew:                           0.205   Prob(JB):                    7.39e-219
Kurtosis:                       1.077   Cond. No.                         3.17
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [18]:
df_model = df[['yes', 'guest_black', 'guest_male', 'name_by_city']].dropna()
model = smf.ols('yes ~ guest_male', data=df_model)
result = model.fit(cov_type='cluster', cov_kwds={'groups': df_model['name_by_city']})
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    yes   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     5.919
Date:                Tue, 11 Aug 2020   Prob (F-statistic):             0.0167
Time:                        21:20:34   Log-Likelihood:                -4485.5
No. Observations:                6235   AIC:                             8975.
Df Residuals:                    6233   BIC:                             8988.
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4706      0.015     32.115      0.000       0.442       0.499
guest_male    -0.0453      0.019     -2.433      0.015      -0.082      -0.009
==============================================================================
Omnibus:                    22284.364   Durbin-Watson:                   1.955
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1031.108
Skew:                           0.207   Prob(JB):                    1.25e-224
Kurtosis:                       1.051   Cond. No.                         2.59
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [19]:
df_model = df[['yes','guest_male','name_by_city', 'host_gender_M', 'host_race_black']].dropna()
df_model3 = df[['yes','guest_male','name_by_city', 'host_gender_M', 'host_race_black', 'multiple_listings', 'shared_property', 'ten_reviews', 'log_price']].dropna()

model = smf.ols('yes ~ guest_male', data=df_model)
result1 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model['name_by_city']})

model = smf.ols('yes ~ guest_male + host_race_black + host_gender_M', data=df_model)
result2 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model['name_by_city']})

model = smf.ols('yes ~ guest_male + host_race_black + host_gender_M + multiple_listings + shared_property + ten_reviews + log_price', data=df_model3)
result3 = model.fit(cov_type='cluster', cov_kwds={'groups': df_model3['name_by_city']})


tble = summary_col([result1, result2, result3], stars=True, float_format='%.2f', regressor_order=['guest_male', 'host_race_black', 'host_gender_M'], info_dict={'N':lambda x: "{0:d}".format(int(x.nobs))})
tble

,yes I,yes II,yes III
guest_male,-0.05**,-0.05**,-0.05**
,(0.02),(0.02),(0.02)
host_race_black,,0.07***,0.09***
,,(0.02),(0.02)
host_gender_M,,-0.05***,-0.05***
,,(0.01),(0.01)
Intercept,0.47***,0.48***,0.74***
,(0.01),(0.01),(0.07)
R-squared,0.00,0.00,0.03
,0.00,0.01,0.03


In [20]:
# Regress each control on treatment variable
used_controls = ['host_gender_M', 'host_race_black', 'multiple_listings', 'shared_property', 'log_price', 
                     'host_race_black', 'host_gender_M', 'host_gender_F', 'shared_property', 'multiple_listings', 
                     'ten_reviews', 'young', 'price_median', 'black_proportion', 'guest_black',
                     'tract_listings', 'pr_filled', 'baltimore', 'dallas', 'los_angeles', 'sl', 'dc']
used_control_models = {}

print('{0:20} {1:>1}'.format('Control Variable', 'P-Value'))
for control in used_controls:
    model = smf.ols(control + ' ~ guest_male', data=df)
    result = model.fit()
    used_control_models[control] = result
    print('{0:20} {1:>6.3f}'.format(control + ':', result.pvalues[1]))

Control Variable     P-Value
host_gender_M:        0.824
host_race_black:      0.843
multiple_listings:    0.504
shared_property:      0.918
log_price:            0.948
host_race_black:      0.843
host_gender_M:        0.824
host_gender_F:        0.538
shared_property:      0.918
multiple_listings:    0.504
ten_reviews:          0.169
young:                0.980
price_median:         0.484
black_proportion:     0.407
guest_black:          0.408
tract_listings:       0.371
pr_filled:            0.884
baltimore:            0.556
dallas:               0.268
los_angeles:          0.538
sl:                   0.360
dc:                   0.475


In [21]:
bw = df.guest_male.apply(lambda x: True if x==0 else False if x==1 else np.NaN) # True if guest is white
df_fisher = pd.DataFrame({'white': bw, 'yes':df['yes']})
df_fisher.dropna(inplace=True)

bw_array = np.array(df_fisher.white)
yes_array = np.array(df_fisher.yes)
def calc_yes(bw_array, ayes_array):
    '''Given the (altered) index array whether individual is black or white (white=True)
    and given the original acception (yes) array, calculate the sum of callbacks for each group.'''
    white_yes= sum(np.multiply(bw_array, yes_array))
    black_yes = sum(np.multiply(np.logical_not(bw_array), yes_array))
    return white_yes, black_yes

n_white = bw.value_counts()[True]
n_black = bw.value_counts()[False]

def fisher_teststat(y0, y1, n0=n_black, n1=n_white):
    return y1/n1 - y0/n0 


wcb, bcb = calc_yes(bw_array, yes_array)
original_teststat = fisher_teststat(bcb, wcb)
original_teststat

0.051649818619154075

In [22]:
rng = np.random.default_rng()
m = 10000
count_le = 0 # Used for counting whether alternative test statistic is larger than orginal test statistic

start_time = time.perf_counter()
for i in range(0, m):
    alt_perm = rng.permutation(bw_array) # Creates random alternative permutation
    wcb, bcb = calc_yes(alt_perm, yes_array) # Sum of callbacks for groups
    alt_t = fisher_teststat(bcb, wcb) # Fisher test for alternative permutation
    if alt_t >= original_teststat: 
        count_le += 1
        
total_time = time.perf_counter() - start_time
print('For m={}, {:.2f} minutes were used '.format(m, total_time/60))
print('Number of simulated test statistics larger or equal to original test statistic:\t', count_le)

For m=10000, 0.13 minutes were used 
Number of simulated test statistics larger or equal to original test statistic:	 4
